# Social Media Monitoring

## Organization List


The following table contains data on the organizations and their associated social acounts

In [ ]:
import qgrid
from smm_wrapper import SMM
from wikiwho_wrapper import WikiWho

qgrid.set_grid_option('maxVisibleRows', 5)

# Create an instance to the smm wrapper
smm = SMM(unit = "organizations")

# Request the politicians from the API
df = smm.dv.get_organizations()

# Show the result
qgrid.show_grid(df)

## Instructions

Please choose an organization from the table above and input the `organization_id` and `aggregation` variables in the cell below:

In [ ]:
import pandas as pd
import datetime

organization_id = 440
aggregation = 'month' #aggregation can be by day ('day'), week ('week') or month('month')

# using api to extract data
wiki_chobs = smm.dv.wikipedia(_id=organization_id, from_date='2017-09-01', to_date='2018-12-31', aggregate_by=aggregation)
wiki_ids = df.loc[df.index == organization_id, 'wp_ids'].iloc[0]

#using wikiwho to extract conflicts and revisions
ww = WikiWho(lng='de')
wiki_data = pd.concat(ww.dv.edit_persistence(page_id=wiki_id) for wiki_id in wiki_ids)
wiki_data['undos'] = wiki_data['dels'] + wiki_data['reins']
wiki_data = wiki_data[['year_month','conflicts','revisions', 'undos']]

In [ ]:
#aggregating the data from wikiwho and grouping it together with chobs
wiki_data = wiki_data.groupby('year_month').agg(
    {
        'conflicts': 'sum',
        'revisions': 'sum',
        'undos': 'sum'
    }).reset_index()
wiki_total = pd.merge(wiki_chobs, wiki_data, left_on=wiki_chobs['date'], right_on=pd.to_datetime(wiki_data['year_month'])).set_index('_id')

## Plotting

In [ ]:
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_total['date'], y=wiki_total['chobs'], name='Change objects'),
            go.Scatter(x=wiki_total['date'], y=wiki_total['revisions'], name='Revisions')], 
    "layout": go.Layout(hovermode='closest', title='Wikipedia (chobs and revisions)', xaxis={'title':'Date'}, yaxis={'title':'N'}, showlegend=True)
})

plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_total['date'], y=wiki_total['undos'], name='Undos'),
            go.Scatter(x=wiki_total['date'], y=wiki_total['conflicts'], name='Conflicts')], 
    "layout": go.Layout(hovermode='closest', title='Wikipedia (undos and conflicts)', xaxis={'title':'Date'}, yaxis={'title':'N'}, showlegend=True)
})